In [28]:
%reload_ext autoreload
%autoreload 2

In [29]:
import bpy
import sys
import tempfile
import typst
from pathlib import Path

project_root = Path.home() / "projects/blender_typst_importer/"
sys.path.append(str(project_root))


In [70]:
from typst_importer.typst_to_svg import typst_express
from typst_importer.notebook_utils import display_svg

def eq(equation: str, name: str, color: str, position=(0, 0, 0), show_indices=False):

    typst_code = f"""
    #set text({color})
    $ {equation} $
    """
    collection = typst_express(
        typst_code,
        origin_to_char=True,
        convert_to_mesh=True,
        name=name,
        position=position,
        show_indices=show_indices
    )
    display_svg(collection.processed_svg, width="230px")
    return collection
imported_collection = eq("a = b + c +g+ d+ d+d+ d+ f+ f", "Start", "aqua", (0, 0, 0.1), show_indices=False)
# eq("a = b + c", "End", "olive", (0, -0.8, 0));

In [72]:
def add_indices_to_collection(imported_collection):
    """
    Add index labels to objects in a collection.
    
    Args:
        imported_collection: The collection containing objects to be indexed.
    
    Returns:
        The indices collection if created, otherwise None.
    """
    # Create a new collection for indices if there are multiple objects
    if len(imported_collection.objects) > 1:
        indices_collection = bpy.data.collections.new(f"{imported_collection.name}_Indices")
        bpy.context.scene.collection.children.link(indices_collection)
        
        for i, obj in enumerate(imported_collection.objects):
            # Create text object at the same location as the curve/mesh
            bpy.ops.object.text_add(location=(0,0,0))
            text_obj = bpy.context.active_object
            text_obj.data.body = str(i)
            text_obj.name = f"Index_{i}"
            bpy.ops.object.origin_set(type="ORIGIN_GEOMETRY", center="MEDIAN")
            text_obj.scale = (0.2, 0.2, 0.2)

            # Make the text smaller
            
            # Set text color to red
            if "Index_Material" not in bpy.data.materials:
                mat = bpy.data.materials.new("Index_Material")
                mat.diffuse_color = (1.0, 0.0, 0.0, 1.0)  # Red color
            else:
                mat = bpy.data.materials["Index_Material"]
            
            if text_obj.data.materials:
                text_obj.data.materials[0] = mat
            else:
                text_obj.data.materials.append(mat)
            
            # Set text position to obj.location with slight z offset
            text_obj.location = (obj.location[0], obj.location[1], obj.location[2] + 0.1)
            
            # Create background circle for the text
            bpy.ops.mesh.primitive_circle_add(vertices=32, radius=0.1, fill_type='NGON', 
                                             location=(obj.location[0], obj.location[1], obj.location[2] + 0.05))
            circle_obj = bpy.context.active_object
            circle_obj.name = f"Index_Bg_{i}"
            
            # Create white material for background with transparency
            if "Index_Bg_Material" not in bpy.data.materials:
                bg_mat = bpy.data.materials.new("Index_Bg_Material")
                bg_mat.diffuse_color = (1.0, 1.0, 1.0, 0.2)  # White with some transparency
                
                # Set up material for transparency
                bg_mat.use_nodes = True
                nodes = bg_mat.node_tree.nodes
                links = bg_mat.node_tree.links
                
                # Clear existing nodes
                for node in nodes:
                    nodes.remove(node)
                
                # Create necessary nodes
                output_node = nodes.new(type="ShaderNodeOutputMaterial")
                output_node.location = (400, 0)
                
                principled_node = nodes.new(type="ShaderNodeBsdfPrincipled")
                principled_node.location = (0, 0)
                principled_node.inputs["Base Color"].default_value = (1.0, 1.0, 1.0, 1.0)
                principled_node.inputs["Alpha"].default_value = 0.2
                
                links.new(principled_node.outputs["BSDF"], output_node.inputs["Surface"])
                
                # Enable transparency settings for Eevee
                bg_mat.blend_method = 'BLEND'
                bg_mat.use_backface_culling = False
            else:
                bg_mat = bpy.data.materials["Index_Bg_Material"]
            
            if circle_obj.data.materials:
                circle_obj.data.materials[0] = bg_mat
            else:
                circle_obj.data.materials.append(bg_mat)
            
            # Move from scene collection to indices collection
            bpy.context.scene.collection.objects.unlink(text_obj)
            indices_collection.objects.link(text_obj)
            
            bpy.context.scene.collection.objects.unlink(circle_obj)
            indices_collection.objects.link(circle_obj)
        
        return indices_collection
    
    return None

indices_collection = add_indices_to_collection(imported_collection)